# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [36]:
# Write your code below.
%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [37]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [38]:
import os
from glob import glob

# Write your code below.
price_data = os.getenv('PRICE_DATA')
px_parquets =  glob(os.path.join(price_data, "**/*.parquet"), recursive = True)

px_parquets, len(px_parquets) 

(['../../05_src/data/prices/BKTI/BKTI_2012/part.0.parquet',
  '../../05_src/data/prices/BKTI/BKTI_2012/part.1.parquet',
  '../../05_src/data/prices/BKTI/BKTI_2015/part.0.parquet',
  '../../05_src/data/prices/BKTI/BKTI_2015/part.1.parquet',
  '../../05_src/data/prices/BKTI/BKTI_2014/part.0.parquet',
  '../../05_src/data/prices/BKTI/BKTI_2014/part.1.parquet',
  '../../05_src/data/prices/BKTI/BKTI_2013/part.0.parquet',
  '../../05_src/data/prices/BKTI/BKTI_2013/part.1.parquet',
  '../../05_src/data/prices/BKTI/BKTI_1980/part.0.parquet',
  '../../05_src/data/prices/BKTI/BKTI_1980/part.1.parquet',
  '../../05_src/data/prices/BKTI/BKTI_1987/part.0.parquet',
  '../../05_src/data/prices/BKTI/BKTI_1987/part.1.parquet',
  '../../05_src/data/prices/BKTI/BKTI_1989/part.0.parquet',
  '../../05_src/data/prices/BKTI/BKTI_1989/part.1.parquet',
  '../../05_src/data/prices/BKTI/BKTI_1988/part.0.parquet',
  '../../05_src/data/prices/BKTI/BKTI_1988/part.1.parquet',
  '../../05_src/data/prices/BKTI/BKTI_19

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [39]:
# Write your code below.
import pandas as pd

dd_px = dd.read_parquet(px_parquets).set_index("ticker")

dd_shift = (
    dd_px
        .groupby('ticker', group_keys=False)
        .apply(
            lambda x: x.sort_values('Date', ascending = True)
                       .assign(
                            Close_lag_1 = x['Close'].shift(1),
                            Adj_Close_lag_1 = x['Adj Close'].shift(1)
                       ), 
            meta = pd.DataFrame(data ={'Date': 'datetime64[ns]',
                    'Open': 'f8',
                    'High': 'f8',
                    'Low': 'f8',
                    'Close': 'f8',
                    'Adj Close': 'f8',
                    'Volume': 'i8',
                    'source': 'object',
                    'Year': 'int32',
                    'Close_lag_1': 'f8',
                    'Adj_Close_lag_1':'f8'},
                    index = pd.Index([], dtype=pd.StringDtype(), name='ticker'))
    ))

dd_feat = dd_shift.assign(
    returns = (dd_shift['Close'] / dd_shift['Close_lag_1']) - 1,
    hi_lo_range = dd_shift['High'] - dd_shift['Low']
)

dd_feat.head()


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,
A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,NaN,NaN,NaN,7.153078
A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,31.473534,27.068665,-0.082386,2.280043
A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,28.880543,24.838577,0.089783,2.816525
A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,31.473534,27.068665,-0.090909,2.592991
A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,28.612303,24.607880,0.026563,1.385908


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [40]:
# Write your code below.
pd_feat = dd_feat.reset_index().compute()
pd_feat = pd_feat.sort_values(["ticker", "Date"])
pd_feat["returns_10days_avg"] = (
    pd_feat.groupby("ticker")["returns"]
            .transform(lambda s: s.rolling(window=10, min_periods=10).mean())
)

pd_feat.head(20)

,ticker,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_10days_avg
0,A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,NaN,NaN,NaN,7.153078,NaN
1,A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,31.473534,27.068665,-0.082386,2.280043,NaN
2,A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,28.880543,24.838577,0.089783,2.816525,NaN
3,A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,31.473534,27.068665,-0.090909,2.592991,NaN
4,A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,28.612303,24.607880,0.026563,1.385908,NaN
5,A,1999-11-26,29.238197,29.685265,29.148785,29.461731,25.338428,1729400.0,A.csv,1999,29.372318,25.261524,0.003044,0.536480,NaN
6,A,1999-11-29,29.327610,30.355865,29.014664,30.132332,25.915169,4074700.0,A.csv,1999,29.461731,25.338428,0.022762,1.341202,NaN
7,A,1999-11-30,30.042919,30.713520,29.282904,30.177038,25.953619,4310000.0,A.csv,1999,30.132332,25.915169,0.001484,1.430616,NaN
8,A,1999-12-01,30.177038,31.071173,29.953505,30.713520,26.415012,2957300.0,A.csv,1999,30.177038,25.953619,0.017778,1.117668,NaN
9,A,1999-12-02,31.294706,32.188843,30.892345,31.562946,27.145563,3069800.0,A.csv,1999,30.713520,26.415012,0.027656,1.296497,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Dask dataframe also provide rolling.mean feature so it is not necessary to convert to pandas to calculate the moving average return.

For our dataset size, Dask may not work better than the panda dataframe. It is because our dataset size is able to fit into RAM and panda will perform this rolling.mean faster than the dask and dask is requird to handle the boundary case in some scenarios as well. Therefore, it should be good choice to use panada in this case.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.